In [2]:
import tensorflow as tf           # create graph # run graph (sess.run)
import keras.backend as K

In [55]:
# constant cannot modify
a = tf.constant('Hello')
bt, bk = tf.constant(10), K.constant(10)
ct, ck = tf.constant(10.), K.constant(10.)

print(a)
print(bt, bk)
print(ct, ck)
print() # keras has float type only

print(type(a))
print(type(bt), type(bk))
print(type(ct), type(ck))
print() # above types are all same 

with tf.Session() as sess:
    print(sess.run(a), type(sess.run(a)))        # bytes
    print(sess.run(bt), type(sess.run(bt)))
    print(sess.run(ct), type(sess.run(ct)))
    print(sess.run(b+b), sess.run(tf.add(b,b)) ) # add for tf only, furthermore, int + float (X)
    print( sess.run([a,bt,ct]), type(sess.run([a,bt,ct])) )     # list

Tensor("Const_98:0", shape=(), dtype=string)
Tensor("Const_99:0", shape=(), dtype=int32) Tensor("Const_100:0", shape=(), dtype=float32)
Tensor("Const_101:0", shape=(), dtype=float32) Tensor("Const_102:0", shape=(), dtype=float32)

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'> <class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'> <class 'tensorflow.python.framework.ops.Tensor'>

b'Hello' <class 'bytes'>
10 <class 'numpy.int32'>
10.0 <class 'numpy.float32'>
[[ 8 10 12]
 [14 16 18]] [[ 8 10 12]
 [14 16 18]]
[b'Hello', 10, 10.0] <class 'list'>


In [58]:
a = tf.constant([1,2,3])
b = tf.constant([4,5,6])

with tf.Session() as sess:
    print(sess.run(a+b), type(sess.run(tf.add(a,b))) )
    print(sess.run(tf.reduce_sum(a)), type(sess.run(tf.reduce_sum(a))) )
    print(sess.run(K.sum(a)), type(sess.run(K.sum(a))) )

[5 7 9] <class 'numpy.ndarray'>
6 <class 'numpy.int32'>
6 <class 'numpy.int32'>


In [45]:
a = tf.constant([[1,2,3],[4,5,6]])
print(a.shape)

with tf.Session() as sess:
    print( sess.run(tf.reduce_sum(a+1,axis=0)), sess.run(tf.reduce_sum(a+1,axis=1)) )
    print( sess.run(K.sum(a+1,axis=0)), sess.run(K.sum(a+1,axis=1)) )
    print( sess.run(tf.reduce_sum(a+1,axis=-1)), sess.run(K.sum(a+1,axis=-1)) )

(2, 3)
[ 7  9 11] [ 9 18]
[ 7  9 11] [ 9 18]
[ 9 18] [ 9 18]


In [59]:
at, ak = tf.random_uniform([3]), K.random_uniform([3])
bt, bk = tf.random_normal([3]), K.random_normal([3])
#ct, ck = tf.zeros(5), K.zeros(5)
#dt, dk = tf.zeros((2,3)), K.zeros((2,3)) # can not print

with tf.Session() as sess:
    print( sess.run(at), sess.run(ak) )
    print( sess.run(bt), sess.run(bk) )

[0.2630545  0.39240968 0.92797244] [0.34468806 0.65074766 0.62426996]
[-1.2628601  -0.10351779  0.1374308 ] [-0.6291197  -0.44383928 -2.2052107 ]


In [68]:
# placeholder cannot initialize
C = tf.placeholder(tf.int64)
D = tf.placeholder(tf.int64)
E = tf.placeholder(tf.int64)
adder = D + E

with tf.Session() as sess:
    for i in range(5):
        print(sess.run(C, {C:i}), end=' ')
    
    print( sess.run(adder, {C:10, D:20, E:30}) )    # 可以一次填充所有的占位符 # 50
    print( sess.run(adder, {D:20, E:30}) )          # 或者只填充計算所需要的 # 50
    #print( sess.run(adder, {E:30} )                # Error! 每次執行sess.run()都要填充
    print( sess.run(adder, {D:[1,3], E:[2,8]}) )    # type error

0 1 2 3 4 50
50
[ 3 11]


In [72]:
# variable must initialize and can be reassigned
W = tf.Variable([.3]) # Capital
b = K.variable([-.3]) # lower
x = tf.placeholder(tf.float32)
linear_model = W * x + b

y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

with tf.Session() as sess:
    sess.run( tf.global_variables_initializer() )
    print(sess.run(linear_model, {x:[1,2,3,4]}))
    print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

    fixW = tf.assign(W, [-1.]) # deep copy fixW 
    fixb = tf.assign(b, [1.])  # deep copy fixb
    sess.run([fixW, fixb])     # replace W as fixW and b as fixb for all functions 
    print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

[0.         0.3        0.6        0.90000004]
23.66
0.0


In [74]:
# linear regression example

# 模型參數
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)

# 輸入與輸出的資料
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss function
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

with tf.Session() as sess:
    sess.run( tf.global_variables_initializer() )

    for i in range(1000):
        sess.run(train, {x:x_train, y:y_train})

    curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x:x_train, y:y_train})
    print("W:{}    b:{}    loss:{}".format(curr_W, curr_b, curr_loss))

W:[-0.9999969]    b:[0.9999908]    loss:5.699973826267524e-11


In [75]:
# specify graph for a session
my_graph = tf.Graph()
with tf.Session(graph=my_graph) as sess:
    x = tf.constant([1,3,6])
    y = tf.constant([1,1,1])
    op = x+y                      # or tf.add(x,y)
    print(sess.run(op))

[2 4 7]
